# Lesson 6: Custom Tools & Functions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harshit-vibes/lyzr-adk-demo/blob/master/notebooks/06_tools_and_functions.ipynb)


**🟡 Intermediate · ⏱ 30 min**

---

Tools let your agent take actions in the real world — call APIs, query databases, run calculations, check inventory. Any Python function can become a tool. The agent decides when and how to call it.

## What you will learn

- Write a Python function as a tool the agent can call
- Add the tool to an agent with `agent.add_tool()`
- Understand why the **docstring** is the most important part of a tool
- Add multiple tools to a single agent and watch it pick the right one
- See the full tool-calling flow from prompt to result

## Prerequisites

Before running this notebook, make sure you have completed:

- **Lesson 1** — Studio & `agent.run()` basics
- **Lesson 2** — Providers and models
- **Lesson 3** — Agent lifecycle
- **Lesson 4** — Structured outputs
- **Lesson 5** — Memory & sessions

You also need your **`LYZR_API_KEY`** set as an environment variable:

```bash
export LYZR_API_KEY="your-api-key-here"
```

In [ ]:
!pip install lyzr-adk[jupyter] -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## 1. What Are Tools?

By default, an agent can only generate text based on its training knowledge. Tools break this limitation by letting the agent **execute code** and bring in real, dynamic data.

A tool is simply a **Python function**. When you attach a function to an agent with `agent.add_tool()`, the underlying LLM gains the ability to call that function during its reasoning process.

Here is what the agent handles automatically:

| Step | What happens |
|------|-------------|
| **1. Decide** | The LLM reads the user message and decides whether a tool is needed |
| **2. Select** | The LLM picks the most appropriate tool based on its docstring |
| **3. Call** | The LLM extracts arguments from the user message and calls the function |
| **4. Incorporate** | The function's return value is fed back to the LLM, which uses it in the final response |

You write the function. The agent handles the rest.

## 2. Writing Your First Tool

The **docstring** is the single most important part of a tool function. The agent reads it to decide:

- **Whether** to call this tool at all
- **When** to call it (what kinds of questions trigger it)
- **What data** to pass as arguments

A vague or missing docstring means the agent will ignore the tool — even if it would have been perfect for the job.

**Type annotations** on parameters are also important. They tell the agent what type of value to extract from the user's message before passing it to the function.

In [ ]:
def get_weather(city: str) -> str:
    """Get the current weather conditions for a given city.

    Returns temperature in Celsius and weather conditions (sunny, cloudy, rainy, etc.)
    """
    # Simulated weather data — in production this would call a real API
    weather_db = {
        "london": "12°C, overcast with light rain",
        "tokyo": "24°C, sunny with light breeze",
        "new york": "18°C, partly cloudy",
        "sydney": "28°C, sunny",
        "paris": "16°C, mostly cloudy",
    }
    city_lower = city.lower()
    if city_lower in weather_db:
        return f"Weather in {city}: {weather_db[city_lower]}"
    return f"Weather data unavailable for {city}. Please check a local weather service."

print("Tool defined!")
print(f"Docstring: {get_weather.__doc__.strip()[:80]}...")

## 3. Adding the Tool to an Agent

Once the function is defined, attaching it to an agent is a single line: `agent.add_tool(my_function)`.

From that point forward, whenever you call `agent.run()`, the agent has access to the tool and will use it when the user's message calls for it. You do not need to tell the agent *when* to call the tool — it figures that out from the docstring and the user's request.

In [ ]:
weather_agent = studio.create_agent(
    name="Weather Assistant",
    provider="openai/gpt-4o",
    role="Weather information specialist",
    goal="Provide accurate weather information using available tools",
    instructions="Always use the get_weather tool when asked about weather. Be conversational."
)

weather_agent.add_tool(get_weather)
print("Tool added to agent!")

# The agent will automatically call get_weather when appropriate
response = weather_agent.run("What's the weather like in Tokyo right now?")
print(response.response)

## 4. Follow-up Questions and Tool Reuse

The same tool can be called multiple times within a single `agent.run()` call. For example, if the user asks to compare two cities, the agent will call `get_weather` twice — once per city — and then synthesize both results into a single coherent answer.

The agent also knows when **not** to call a tool. If a question can be answered from general knowledge (such as packing advice based on already-retrieved weather), it will use the tool result as context rather than re-calling it unnecessarily.

In [ ]:
# The agent decides when to call the tool vs answer from knowledge
r1 = weather_agent.run("Compare the weather in London and Sydney.")
print(f"Comparison: {r1.response}")

r2 = weather_agent.run("What should I pack for a trip to Paris in this weather?")
print(f"\nPacking advice: {r2.response}")

## 5. Multiple Tools

An agent can hold many tools at once. When a user sends a message, the agent scans all available tool docstrings and selects whichever tool (or combination of tools) best fits the request.

This means you can build a **general-purpose assistant** by composing many small, focused tool functions — each doing one thing well — rather than writing one monolithic function that tries to handle everything.

Best practices for multi-tool agents:

- Keep each tool **focused on a single responsibility**
- Make docstrings **specific** — avoid overlap that could confuse the agent
- Include example inputs/outputs in the docstring for complex tools

In [ ]:
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression and return the numeric result.

    Supports basic arithmetic: +, -, *, /, ** (power), and parentheses.
    Example: '(15 + 7) * 3' returns '66'
    """
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return f"Result: {result}"
    except Exception as e:
        return f"Could not evaluate '{expression}': {e}"


def word_count(text: str) -> str:
    """Count the number of words and characters in a given text string."""
    words = len(text.split())
    chars = len(text)
    return f"Words: {words}, Characters: {chars}"


def currency_convert(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert an amount between currencies using approximate exchange rates.

    Supports: USD, EUR, GBP, JPY, AUD, CAD
    """
    rates_to_usd = {
        "USD": 1.0,
        "EUR": 1.08,
        "GBP": 1.27,
        "JPY": 0.0067,
        "AUD": 0.65,
        "CAD": 0.74,
    }
    if from_currency not in rates_to_usd or to_currency not in rates_to_usd:
        return f"Unsupported currency. Available: {', '.join(rates_to_usd.keys())}"
    usd = amount * rates_to_usd[from_currency]
    result = usd / rates_to_usd[to_currency]
    return f"{amount} {from_currency} = {result:.2f} {to_currency}"


# Create a multi-tool agent
utility_agent = studio.create_agent(
    name="Utility Assistant",
    provider="openai/gpt-4o",
    role="General-purpose utility assistant",
    goal="Help with calculations, text analysis, and currency conversions",
    instructions="Use the appropriate tool for each request. Show the tool result in your response."
)

utility_agent.add_tool(calculate)
utility_agent.add_tool(word_count)
utility_agent.add_tool(currency_convert)
print("3 tools added!")

In [ ]:
# Agent picks the right tool automatically
r1 = utility_agent.run("What is (144 / 12) + 7.5?")
print(f"Math: {r1.response}")

r2 = utility_agent.run("How many words are in: 'The quick brown fox jumps over the lazy dog'?")
print(f"\nWord count: {r2.response}")

r3 = utility_agent.run("Convert 250 EUR to Japanese Yen.")
print(f"\nCurrency: {r3.response}")

## Common Mistake: Missing Docstring

The most frequent tool-related mistake is defining a function without a docstring — or with a docstring so vague it gives the agent no useful information.

Without a meaningful docstring, the agent has no way to know:
- What the tool does
- When it should be used
- What arguments to pass

The result: the agent ignores the tool entirely, even when it would have been the perfect choice.

**Rule of thumb:** If a human developer could not understand what the tool does from the docstring alone, the agent cannot either.

In [ ]:
# Bad — no docstring. The agent won't know what this does or when to use it.
def mystery_tool(x: int) -> int:
    return x * 42


# Good — clear docstring. The agent understands and uses this appropriately.
def multiply_by_42(number: int) -> str:
    """Multiply any integer by 42 and return the result.
    Use this when asked to apply the meaning-of-life multiplier."""
    return f"{number} x 42 = {number * 42}"


test_agent = studio.create_agent(
    name="Docstring Test",
    provider="openai/gpt-4o",
    role="Test assistant",
    goal="Test tool calling",
    instructions="Use tools when available."
)

test_agent.add_tool(mystery_tool)
test_agent.add_tool(multiply_by_42)

r = test_agent.run("Apply the meaning-of-life multiplier to 7.")
print(r.response)  # Should use multiply_by_42, not mystery_tool

## Exercise: Shopping Price Checker

Your task is to write a tool that looks up product prices from a catalog, attach it to an agent, and ask the agent to recommend the cheapest option from a list.

**Steps:**

1. Complete the `get_product_price` function — write a clear docstring and implement the lookup logic
2. Fill in the agent configuration with an appropriate name, role, goal, and instructions
3. Run the agent with the provided query and observe how it combines multiple tool calls to compute the total

Fill in the `...` placeholders below.

In [ ]:
# Product catalog (simulated database)
PRODUCT_CATALOG = {
    "laptop": 999.99,
    "headphones": 149.99,
    "keyboard": 79.99,
    "monitor": 349.99,
    "mouse": 29.99,
    "webcam": 89.99,
}


# TODO: Write a get_product_price tool function with a clear docstring
def get_product_price(product_name: str) -> str:
    """..."""  # TODO: Write a clear docstring
    # TODO: Look up the product in PRODUCT_CATALOG and return the price
    # If not found, return a helpful message
    ...


# TODO: Create an agent and add the tool
shopping_agent = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...
)
shopping_agent.add_tool(get_product_price)

# TODO: Ask the agent to find the cheapest item from a list
response = shopping_agent.run(
    "I need a keyboard, mouse, and webcam. Which combination is cheapest? What's the total?"
)
print(response.response)

## Summary

### Tool Best Practices

| Practice | Why it matters |
|----------|----------------|
| Write a clear, specific docstring | The agent reads this to decide when and how to call the tool |
| Add type annotations to parameters | Helps the agent extract and pass the correct data types |
| Return a string or JSON-serializable value | The return value is fed back into the LLM as text |
| Keep each tool focused on one task | Prevents ambiguity when the agent chooses between tools |
| Handle errors gracefully inside the tool | Return a helpful error string rather than raising an exception |
| Use descriptive function names | Names reinforce the docstring and help the agent disambiguate |

### Key Takeaways

- Any Python function can become an agent tool — attach it with `agent.add_tool(fn)`.
- The **docstring** is what the agent reads to decide when to call the tool. A missing or vague docstring means the tool will be ignored.
- **Type annotations** on parameters guide the agent to extract and pass the right data.
- Agents can hold **multiple tools** and will automatically select the most appropriate one per request.
- The tool-calling flow is fully automatic: the agent decides, calls, and incorporates the result — you just write the function.

## Next Steps

You now know how to extend your agents with custom Python tools. In the next lesson, you will connect agents to real knowledge sources.

**Lesson 7: RAG & Knowledge Bases** — give your agent access to documents, PDFs, and knowledge bases so it can answer questions grounded in your own data.

Head to `07_rag_and_knowledge_bases.ipynb` to continue.